In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

SHEET_NAMES = ["Pre & Post", "Weekly", "Daily"]

In [2]:
df = pd.read_csv("data/class_data.csv", encoding = 'latin-1')
df_interested = df[df["pre_studyinterest"] == "Yes"]

In [87]:
# construct possible varnames for each Name Stem in its Construct group
# flag in ["pre", "post", 'd', 'w']
def build_counstruct_variable_dict(codebook, flag):
    appending = [c for c in list(codebook.columns)[1:list(codebook.columns).index("Construct")]]
    if "pre" in appending:
        construct_var_dict = \
            codebook.groupby("Construct").apply(
                lambda d: {i: [a + "_" + i for a in appending if d[d["Name Stem"] == i][a].tolist()[0] == "x"] \
                           for i in d["Name Stem"]}).to_dict()
            
    else:
        construct_var_dict = \
            codebook.groupby("Construct").apply(
                lambda d: {i: [i + flag + str(a) for a in appending if d[d["Name Stem"] == i][a].tolist()[0] == "x"] \
                           for i in d["Name Stem"]}).to_dict()
    return construct_var_dict

# Verify possuble varnames by checking in bio class variables set
def find_construct_variable(df, codebook, flag):
    construct_var_dict = build_counstruct_variable_dict(codebook, flag)
    return {construct: {var_root: [var for var in construct_var_dict[construct][var_root] + [var_root] if var in df.columns] \
                          for var_root in construct_var_dict[construct]} for construct in construct_var_dict}

def print_non_exist_variable(df, to_ret):
    print("Name Stems that are not in df:  ")
    for construct in to_ret:
        for var_root in to_ret[construct]:
            if len(to_ret[construct][var_root]) == 0:
                print([construct, var_root])
    print('\n')

def build_var_null_percentage_df(df, sheet_name, verbal = False, overwrite = True, student_interested = False):
    
    def construct_percentage_df(construct_var_dict):
        df_percentage_master = pd.DataFrame()
        for construct in construct_var_dict:
            found_var = [var for var_root in construct_var_dict[construct] for var in construct_var_dict[construct][var_root]]
            df_construct_sub = df[found_var]
            to_view = pd.DataFrame(df_construct_sub.count()).reset_index()
            to_view.columns = ["var", "count"]
            to_view["null_count"] = df_construct_sub.shape[0] - to_view["count"]
            to_view["null_percentage"] = to_view["null_count"] / df_construct_sub.shape[0]
            to_view["Name Stem"] = [var_root for var_root in construct_var_dict[construct] for var in construct_var_dict[construct][var_root]]
            to_view["Construct"] = construct
            df_percentage_master = pd.concat([df_percentage_master, to_view], axis = 0)
        df_percentage_master["flag"] = [i.replace(j, '').replace('_', '') if i.replace(j, '').replace('_', '') else "original" 
                       for i, j in zip(df_percentage_master["var"], df_percentage_master["Name Stem"])]
        df_percentage_master = pd.DataFrame(df_percentage_master.pivot(
            index='Name Stem', columns="flag", values=["count", "null_count", "null_percentage"]))
        return df_percentage_master
    
    def write_to_excel(df_percentage, df_codebook, sheet_name = sheet_name):
        fname = "data/{}_interested_null_percentage.xlsx".format(sheet_name) if student_interested else \
                "data/{}_null_percentage.xlsx".format(sheet_name)
        appending = [c for c in list(df_codebook.columns)[1:list(df_codebook.columns).index("Construct")]]
        with pd.ExcelWriter(fname) as writer:
            for level in df_percentage.columns.levels[0]:
                df_percentage_level = pd.DataFrame(df_percentage[level]).reset_index()
                if sheet_name == "Daily" or sheet_name == "Weekly":
                    df_percentage_level["mean"] = df_percentage_level[["{}{}".format(sheet_name[0].lower(), i) for i in appending
                                                                      if "{}{}".format(sheet_name[0].lower(), i) 
                                                                       in df_percentage_level.columns]].mean(axis=1)
                df_percentage_level = df_percentage_level.merge(df_codebook[codebook_vars], 
                                                               left_on = "Name Stem", right_on = "Name Stem", how = "left")
                df_percentage_level.to_excel(writer, sheet_name = level)
    
    def print_percentage_head(df_percentage, df_codebook):
        df_percentage.columns = ['_'.join(col[::-1]).strip() for col in df_percentage.columns.values]
        df_percentage = pd.DataFrame(df_percentage).reset_index().merge(df_codebook[codebook_vars], 
                                                                   left_on = "Name Stem", right_on = "Name Stem", how = "left")
        df_percentage = df_percentage[codebook_vars + [c for c in df_percentage.columns if c not in codebook_vars]]
        display(df_percentage.head())
    
    df_codebook =  pd.read_excel("data/codebook.xlsx", sheet_name=sheet_name)
    construct_var_dict = find_construct_variable(df, df_codebook, sheet_name[0].lower())
    df_percentage = construct_percentage_df(construct_var_dict)
    codebook_vars = ["Name Stem", "Construct", "Item", "Response Values", "Label"]
    
    if verbal:
        print(sheet_name, '\n\n')
        print_non_exist_variable(df, construct_var_dict)
        print_percentage_head(df_percentage.copy(), df_codebook)
    
    if overwrite:
        write_to_excel(df_percentage.copy(), df_codebook)
  
    return df_percentage
        

In [89]:
temp = build_var_null_percentage_df(df_interested, "Pre & Post", verbal = True, student_interested=True)

Pre & Post 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Self-Concept of Ability', 'abil_gen_bio1']
['Self-Concept of Ability', 'abil_gen_bio3']
['Self-Concept of Ability', 'abil_gen_chem1']
['Self-Concept of Ability', 'abil_gen_chem3']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,post_count,pre_count,original_null_count,post_null_count,pre_null_count,original_null_percentage,post_null_percentage,pre_null_percentage
0,aca1,Academic Integration,"Talk with faculty about academic matters, outs...",(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty ac...,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN
1,aca2,Academic Integration,Meet with an academic advisor concerning acade...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with academic a...,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN
2,aca3,Academic Integration,Meet with a student mentor concerning course a...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with student me...,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN
3,aca4,Academic Integration,Attend study groups outside of the classroom,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter attending study groups ...,NaN,69.0,NaN,NaN,30.0,NaN,NaN,0.303030,NaN
4,aca5,Academic Integration,Have informal or social contacts with faculty ...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty so...,NaN,69.0,NaN,NaN,30.0,NaN,NaN,0.303030,NaN


In [71]:
temp2 = temp["count"].reset_index()
temp2["mean"] = temp2.mean(axis=1)

In [72]:
temp2

flag,Name Stem,d1,d2,d3,d4,d5,d6,original,mean
0,dadd,NaN,62.0,63.0,62.0,61.0,47.0,NaN,59.000000
1,daddo,NaN,5.0,12.0,6.0,7.0,10.0,NaN,8.000000
2,dcact1,NaN,NaN,NaN,NaN,NaN,NaN,60.0,60.000000
3,dcact10,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.000000
4,dcact11,NaN,NaN,NaN,NaN,NaN,NaN,57.0,57.000000
5,dcact12,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.000000
6,dcact13,NaN,NaN,NaN,NaN,NaN,NaN,22.0,22.000000
7,dcact14,NaN,NaN,NaN,NaN,NaN,NaN,8.0,8.000000
8,dcact15,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.000000
9,dcact16,NaN,NaN,NaN,NaN,NaN,NaN,48.0,48.000000


## Interested Students

In [25]:
for sheet_name in SHEET_NAMES:
    build_var_null_percentage_df(df_interested, sheet_name, verbal = True, student_interested=True)

Pre & Post 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Self-Concept of Ability', 'abil_gen_bio1']
['Self-Concept of Ability', 'abil_gen_bio3']
['Self-Concept of Ability', 'abil_gen_chem1']
['Self-Concept of Ability', 'abil_gen_chem3']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']


Index(['count', 'null_count', 'null_percentage'], dtype='object') 



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,Name Stem,"(Name Stem, )","(count, original)","(count, post)","(count, pre)","(null_count, original)","(null_count, post)","(null_count, pre)","(null_percentage, original)","(null_percentage, post)","(null_percentage, pre)",Construct,Item,Response Values,Label
0,aca1,aca1,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN,Academic Integration,"Talk with faculty about academic matters, outs...",(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty ac...
1,aca2,aca2,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN,Academic Integration,Meet with an academic advisor concerning acade...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with academic a...
2,aca3,aca3,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN,Academic Integration,Meet with a student mentor concerning course a...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with student me...
3,aca4,aca4,NaN,69.0,NaN,NaN,30.0,NaN,NaN,0.303030,NaN,Academic Integration,Attend study groups outside of the classroom,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter attending study groups ...
4,aca5,aca5,NaN,69.0,NaN,NaN,30.0,NaN,NaN,0.303030,NaN,Academic Integration,Have informal or social contacts with faculty ...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty so...


ValueError: No engine for filetype: 'csv'

## All Students

In [ ]:
for sheet_name in SHEET_NAMES:
    build_var_null_percentage_df(df, sheet_name, verbal = True, student_interested=False)